<div class='alert alert-info' style='text-align: center'><h1>Exploring rib suppression on CXR</h1>
    - yet another chest x-ray processing notebook -
</div>

#### It occured to me that I should try to create a rib suppression tool. I've not seen any open source software for it, although there are proprietarty systems with filters such as this.

- The concept is to create masks that can be used to filter out ribs to enable better 'visualization' of underlying lung tissue. 
- Tissue density under the ribs varies greatly depending on underlying anatomy, so we'll use multiple masks to deal with this.
- In this case, I used two masks to demonstrate the thought process.
- This notebook relies on a dataset of 2 binary masks I made by hand specifically for the DICOM image I'm using.

In [ ]:
import numpy as np
import pydicom
import cv2
import sys
import matplotlib.pyplot as plt

- First we'll grab a random DICOM image from the SIIM Covid-19 Detection set, and squeeze the pixels to a range of 0-255.

In [ ]:
# Load the image
image = pydicom.dcmread('../input/siim-covid19-detection/train/00a76543ed93/4a223cccbe04/ad8d4a5ba8f0.dcm')

# Normalize it to 8 bit
image = cv2.normalize(image.pixel_array, None, 0,255, cv2.NORM_MINMAX, cv2.CV_8U)
plt.figure(figsize= (10,5))
plt.imshow(image, cmap='gray');

- I made two binary masks using GIMP. They could be created via segmentation or other means (edge detection, or bit-plane slicing maybe?)
- One of the masks is for the ribs, the other represents the lung fields without the mediastinum.
- From these two masks, we'll create a third mask .. which is the 'difference' of the other two.

In [ ]:
# Load the masks
mask_rib = cv2.imread('../input/cxr-mask/ad8d4a5ba8f0_mask.jpg', 0)
ret,mask_rib = cv2.threshold(mask_rib,127,255,cv2.THRESH_BINARY)

mask_lung = cv2.imread('../input/cxr-mask/ad8d4a5ba8f0_mask_2.jpg', 0)
ret2,mask_lung = cv2.threshold(mask_lung,127,255,cv2.THRESH_BINARY)

# Create the third mask from the first two using CV2 BitwiseOR
mask_diff = cv2.bitwise_or(mask_rib, mask_rib, mask = mask_lung)

# Make inverted versions of the masks
mask_rib_inv = cv2.bitwise_not(mask_rib)
mask_lung_inv = cv2.bitwise_not(mask_lung)
mask_diff_inv = cv2.bitwise_not(mask_diff)

In [ ]:
plt.figure(figsize= (20,10))
plt.subplot(1, 3, 1)
plt.imshow(mask_rib, cmap='gray');
plt.subplot(1, 3, 2)
plt.imshow(mask_lung, cmap='gray');
plt.subplot(1, 3, 3)
plt.imshow(mask_diff, cmap='gray');

- Now we'll get the pixels from the original image using the rib mask and darken them, then put them back onto the image using the inverse mask.

In [ ]:
# Get pixels from the original image using the first mask
pix = cv2.bitwise_or(image, image, mask = mask_rib)

# Darken them slightly
pix = pix * .95
pix = np.where(pix < 1, 0, pix)
 
# Make an inverse mask of the original image using the inverted rib mask
inv = cv2.bitwise_or(image, image, mask = mask_rib_inv)

# Add the two images together
output = pix + inv

In [ ]:
# Display the image steps
plt.figure(figsize= (20,5))
plt.subplot(1, 3, 1)
plt.imshow(pix, cmap='gray');
plt.subplot(1, 3, 2)
plt.imshow(inv, cmap='gray');
plt.subplot(1, 3, 3)
plt.imshow(output, cmap='gray');

- I only darkened the masked pixels slightly, but you can see that the ribs over the heart are slightly less visible. (hard to see in this small image).
- Next, we'll make a mask of the masked pixels from the last step, and darken them a little bit.
- This will darken only the pixels in the lung area and exclude the heart and mediastinum.

In [ ]:
# Get the pixels from the first mapped set using the second mask
pix2 = cv2.bitwise_or(pix, pix, mask = mask_lung)

# Darken them a bit, I randomly picked this value for demonstration purposes
pix2 = pix2 * .82
pix2 = np.where(pix2 < 1, 0, pix2)
 
# Make an inverse mask of the output of the first masking operation
inv2 = cv2.bitwise_or(output, output, mask = mask_diff_inv)

# Add the third mask mapped pixels and the inverse mask back together
output2 = pix2 + inv2

In [ ]:
# Display the steps
plt.figure(figsize= (20,5))
plt.subplot(1, 3, 1)
plt.imshow(pix2, cmap='gray');
plt.subplot(1, 3, 2)
plt.imshow(inv2, cmap='gray');
plt.subplot(1, 3, 3)
plt.imshow(output2, cmap='gray');

In [ ]:
# Display the original image and the final image
plt.figure(figsize= (20,10))
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray');
plt.subplot(1, 2, 2)
plt.imshow(output2, cmap='gray');

#### Conclusion
- Of course this is a quick idea and my results aren't very good but, you can see in the final image on the right that the ribs are much less prominent than they are in the original image.
- The soft tissue of the lung has a little less noise in it and the underlying pixel values haven't been disturbed.
- With a few more masks and some filtering (maybe edge detection for the denser periosteum on the rib edges?), this could be useful.

**Here are some other processing notebooks I made:**
- Lung segmentation without CNN -> https://www.kaggle.com/davidbroberts/lung-segmentation-without-cnn
- Applying filters to x-rays -> https://www.kaggle.com/davidbroberts/applying-filters-to-chest-x-rays
- Manual DICOM VOI LUT -> https://www.kaggle.com/davidbroberts/manual-dicom-voi-lut
- Apply Unsharp Mask to Chest X-Rays -> https://www.kaggle.com/davidbroberts/unsharp-masking-chest-x-rays
- Cropping Chest X-Rays -> https://www.kaggle.com/davidbroberts/cropping-chest-x-rays
- Bounding Boxes on Cropped Images -> https://www.kaggle.com/davidbroberts/bounding-boxes-on-cropped-images
- Visualizing Chest X-Ray bit planes -> https://www.kaggle.com/davidbroberts/visualizing-chest-x-ray-bitplanes
- DICOM full range pixels as CNN input -> https://www.kaggle.com/davidbroberts/dicom-full-range-pixels-as-cnn-input
- Standardizing Chest X-Ray Dataset Exports -> https://www.kaggle.com/davidbroberts/standardizing-cxr-datasets